In [2]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility
from math import sqrt

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import scipy.io
mat = scipy.io.loadmat('/home/aniruddha/Documents/data_10feature.mat')

arr = mat['TR1_10feature']
arr = np.array(arr)
arr = arr.reshape(-1)
print(arr.shape)
X_train = []
for i in range(0,14):
    for j in range(0,arr[i].shape[0]):
        X_train.append(arr[i][j])
X_train = np.array(X_train)
print(X_train.shape)
y_train = []
for i in range(0,arr.shape[0]):
    for j in range(0,arr[i].shape[0]):
        y_train.append(i)
y_train = np.array(y_train)
print(y_train.shape)
print(y_train[1])

arr1 = mat['TS1_10feature']
arr1 = np.array(arr1)
arr1 = arr1.reshape(-1)
print(arr1.shape)
X_test = []
for i in range(0,14):
    for j in range(0,arr1[i].shape[0]):
        X_test.append(arr1[i][j])
X_test = np.array(X_test)
print(X_test.shape)
y_test = []
for i in range(0,arr1.shape[0]):
    for j in range(0,arr1[i].shape[0]):
        y_test.append(i)
y_test = np.array(y_test)
print(y_test.shape)
print(y_test[1])

arr2 = mat['TS2_10feature']
arr2 = np.array(arr2)
arr2 = arr2.reshape(-1)
print(arr2.shape)
X_test1 = []
for i in range(0,14):
    for j in range(0,arr2[i].shape[0]):
        X_test1.append(arr2[i][j])
X_test1 = np.array(X_test1)
print(X_test1.shape)
y_test1 = []
for i in range(0,arr2.shape[0]):
    for j in range(0,arr2[i].shape[0]):
        y_test1.append(i)
y_test1 = np.array(y_test1)
print(y_test1.shape)
print(y_test1[1])


(14,)
(1242, 10)
(1242,)
0
(14,)
(1252, 10)
(1252,)
0
(14,)
(627, 10)
(627,)
0


In [4]:
print(X_train.max())
print(X_test.max())
print(X_test1.max())

6707
6804
7890


In [5]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_test1 = X_test1.astype('float32')
X_train = X_train/10000
X_test = X_test/10000
X_test1 = X_test1/10000
print(X_train.max())
print(X_test.max())
print(X_test1.max())

0.6707
0.6804
0.789


In [6]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state = 0)
X_test, y_test = shuffle(X_test, y_test, random_state=0)
X_test1, y_test1 = shuffle(X_test1, y_test1, random_state=0)

In [7]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 8 # 1st layer num features
num_hidden_2 = 6 # 2nd layer num features (the latent dim)
num_input = 10 
num_classes = 14

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_uniform([num_input, num_hidden_1], minval=-4*np.sqrt(6.0/(num_input + num_hidden_1)), maxval=4*np.sqrt(6.0/(num_input + num_hidden_1)))),
    'encoder_h2': tf.Variable(tf.random_uniform([num_hidden_1, num_hidden_2], minval=-4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)), maxval=4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)))),
    'decoder_h1': tf.Variable(tf.random_uniform([num_hidden_2, num_hidden_1], minval=-4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)), maxval=4*np.sqrt(6.0/(num_hidden_1 + num_hidden_2)))),
    'decoder_h2': tf.Variable(tf.random_uniform([num_hidden_1, num_input], minval=-4*np.sqrt(6.0/(num_input + num_hidden_1)), maxval=4*np.sqrt(6.0/(num_input + num_hidden_1)))),
    'classifier1_h': tf.Variable(tf.random_uniform([num_hidden_2, 10], minval=-4*np.sqrt(6.0/(10 + num_hidden_2)), maxval=4*np.sqrt(6.0/(10 + num_hidden_2)))),
    'classifier_h': tf.Variable(tf.random_uniform([10, num_classes], minval=-4*np.sqrt(6.0/(10 + num_classes)), maxval=4*np.sqrt(6.0/(10 + num_classes)))),
}
biases = {
    'encoder_b1': tf.Variable(tf.truncated_normal([num_hidden_1])/sqrt(num_hidden_1)),
    'encoder_b2': tf.Variable(tf.truncated_normal([num_hidden_2])/sqrt(num_hidden_2)),
    'decoder_b1': tf.Variable(tf.truncated_normal([num_hidden_1])/sqrt(num_hidden_1)),
    'decoder_b2': tf.Variable(tf.truncated_normal([num_input])/sqrt(num_hidden_2)),
    'classifier1_b': tf.Variable(tf.truncated_normal([10])/sqrt(10)),
    'classifier_b': tf.Variable(tf.truncated_normal([num_classes])/sqrt(num_classes)),
}

In [8]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
classify1 = tf.nn.sigmoid(tf.add(tf.matmul(encoder_op, weights['classifier1_h']), biases['classifier1_b']))
label_pred = tf.nn.softmax(tf.add(tf.matmul(classify1, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [9]:
from keras.utils import np_utils
y_test1 = np_utils.to_categorical(y_test1)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test1.shape)

(1242, 14)
(627, 14)


In [11]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [20]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 32
num_batch = 38

# Training
for i in range(0,2000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 38

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))
    print("Epoch:", (i + 1), "accuracy =", "{:.8f}".format(sess.run(accuracy, feed_dict={X: X_train, Y: y_train})))

Epoch: 1 cost = 0.26867121
Epoch: 1 accuracy = 0.10869565
Epoch: 2 cost = 0.26723198
Epoch: 2 accuracy = 0.12882447
Epoch: 3 cost = 0.26570786
Epoch: 3 accuracy = 0.14090177
Epoch: 4 cost = 0.26410151
Epoch: 4 accuracy = 0.14412238
Epoch: 5 cost = 0.26241683
Epoch: 5 accuracy = 0.13929147
Epoch: 6 cost = 0.26065880
Epoch: 6 accuracy = 0.12640902
Epoch: 7 cost = 0.25883327
Epoch: 7 accuracy = 0.11191627
Epoch: 8 cost = 0.25694661
Epoch: 8 accuracy = 0.10225443
Epoch: 9 cost = 0.25500535
Epoch: 9 accuracy = 0.08695652
Epoch: 10 cost = 0.25301576
Epoch: 10 accuracy = 0.07648953
Epoch: 11 cost = 0.25098356
Epoch: 11 accuracy = 0.06924316
Epoch: 12 cost = 0.24891376
Epoch: 12 accuracy = 0.06360708
Epoch: 13 cost = 0.24681047
Epoch: 13 accuracy = 0.06280193
Epoch: 14 cost = 0.24467704
Epoch: 14 accuracy = 0.06199678
Epoch: 15 cost = 0.24251645
Epoch: 15 accuracy = 0.06199678
Epoch: 16 cost = 0.24033139
Epoch: 16 accuracy = 0.06199678
Epoch: 17 cost = 0.23812472
Epoch: 17 accuracy = 0.0619967

Epoch: 183 cost = 0.13150418
Epoch: 183 accuracy = 0.28502417
Epoch: 184 cost = 0.13115644
Epoch: 184 accuracy = 0.28502417
Epoch: 185 cost = 0.13087302
Epoch: 185 accuracy = 0.28582931
Epoch: 186 cost = 0.13069569
Epoch: 186 accuracy = 0.28582931
Epoch: 187 cost = 0.13083142
Epoch: 187 accuracy = 0.28582931
Epoch: 188 cost = 0.13097082
Epoch: 188 accuracy = 0.28502417
Epoch: 189 cost = 0.13032861
Epoch: 189 accuracy = 0.28582931
Epoch: 190 cost = 0.12995838
Epoch: 190 accuracy = 0.28582931
Epoch: 191 cost = 0.12972627
Epoch: 191 accuracy = 0.28582931
Epoch: 192 cost = 0.12953197
Epoch: 192 accuracy = 0.28582931
Epoch: 193 cost = 0.12935805
Epoch: 193 accuracy = 0.28663447
Epoch: 194 cost = 0.12923941
Epoch: 194 accuracy = 0.28743961
Epoch: 195 cost = 0.12928808
Epoch: 195 accuracy = 0.28824475
Epoch: 196 cost = 0.12923943
Epoch: 196 accuracy = 0.28743961
Epoch: 197 cost = 0.12877391
Epoch: 197 accuracy = 0.28663447
Epoch: 198 cost = 0.12842306
Epoch: 198 accuracy = 0.28904992
Epoch: 1

Epoch: 361 accuracy = 0.44847021
Epoch: 362 cost = 0.10547221
Epoch: 362 accuracy = 0.44847021
Epoch: 363 cost = 0.10494482
Epoch: 363 accuracy = 0.46135265
Epoch: 364 cost = 0.10417346
Epoch: 364 accuracy = 0.46376812
Epoch: 365 cost = 0.10381388
Epoch: 365 accuracy = 0.46054751
Epoch: 366 cost = 0.10360556
Epoch: 366 accuracy = 0.46296296
Epoch: 367 cost = 0.10343920
Epoch: 367 accuracy = 0.46296296
Epoch: 368 cost = 0.10328463
Epoch: 368 accuracy = 0.46537843
Epoch: 369 cost = 0.10313330
Epoch: 369 accuracy = 0.47020933
Epoch: 370 cost = 0.10298880
Epoch: 370 accuracy = 0.47504026
Epoch: 371 cost = 0.10294353
Epoch: 371 accuracy = 0.47101450
Epoch: 372 cost = 0.10348660
Epoch: 372 accuracy = 0.46779388
Epoch: 373 cost = 0.10340344
Epoch: 373 accuracy = 0.47826087
Epoch: 374 cost = 0.10240156
Epoch: 374 accuracy = 0.47584540
Epoch: 375 cost = 0.10209691
Epoch: 375 accuracy = 0.47101450
Epoch: 376 cost = 0.10234331
Epoch: 376 accuracy = 0.46376812
Epoch: 377 cost = 0.10291985
Epoch: 3

Epoch: 500 accuracy = 0.55877614
Epoch: 501 cost = 0.08306023
Epoch: 501 accuracy = 0.56602252
Epoch: 502 cost = 0.08318955
Epoch: 502 accuracy = 0.56843799
Epoch: 503 cost = 0.08289079
Epoch: 503 accuracy = 0.57326895
Epoch: 504 cost = 0.08168919
Epoch: 504 accuracy = 0.57971013
Epoch: 505 cost = 0.08121600
Epoch: 505 accuracy = 0.58293074
Epoch: 506 cost = 0.08108154
Epoch: 506 accuracy = 0.58776170
Epoch: 507 cost = 0.08114982
Epoch: 507 accuracy = 0.59017712
Epoch: 508 cost = 0.08135552
Epoch: 508 accuracy = 0.59017712
Epoch: 509 cost = 0.08129777
Epoch: 509 accuracy = 0.59098226
Epoch: 510 cost = 0.08099977
Epoch: 510 accuracy = 0.59500808
Epoch: 511 cost = 0.08074869
Epoch: 511 accuracy = 0.58937198
Epoch: 512 cost = 0.08069005
Epoch: 512 accuracy = 0.58776170
Epoch: 513 cost = 0.08070247
Epoch: 513 accuracy = 0.58293074
Epoch: 514 cost = 0.08052182
Epoch: 514 accuracy = 0.57971013
Epoch: 515 cost = 0.08092367
Epoch: 515 accuracy = 0.56360710
Epoch: 516 cost = 0.08182476
Epoch: 5

Epoch: 718 cost = 0.06425492
Epoch: 718 accuracy = 0.66747183
Epoch: 719 cost = 0.06387021
Epoch: 719 accuracy = 0.67069244
Epoch: 720 cost = 0.06382072
Epoch: 720 accuracy = 0.64975846
Epoch: 721 cost = 0.06429308
Epoch: 721 accuracy = 0.61835748
Epoch: 722 cost = 0.06726533
Epoch: 722 accuracy = 0.63123995
Epoch: 723 cost = 0.06570932
Epoch: 723 accuracy = 0.65217394
Epoch: 724 cost = 0.06428841
Epoch: 724 accuracy = 0.66022545
Epoch: 725 cost = 0.06374097
Epoch: 725 accuracy = 0.65700483
Epoch: 726 cost = 0.06346136
Epoch: 726 accuracy = 0.65942031
Epoch: 727 cost = 0.06334623
Epoch: 727 accuracy = 0.66103059
Epoch: 728 cost = 0.06329743
Epoch: 728 accuracy = 0.66103059
Epoch: 729 cost = 0.06331754
Epoch: 729 accuracy = 0.65619969
Epoch: 730 cost = 0.06359762
Epoch: 730 accuracy = 0.65378422
Epoch: 731 cost = 0.06486053
Epoch: 731 accuracy = 0.63607085
Epoch: 732 cost = 0.06564215
Epoch: 732 accuracy = 0.63526571
Epoch: 733 cost = 0.06397065
Epoch: 733 accuracy = 0.64734298
Epoch: 7

...

Epoch: 1247 accuracy = 0.70933980
Epoch: 1248 cost = 0.04411021
Epoch: 1248 accuracy = 0.70209342
Epoch: 1249 cost = 0.04420088
Epoch: 1249 accuracy = 0.69243157
Epoch: 1250 cost = 0.04510755
Epoch: 1250 accuracy = 0.68276972
Epoch: 1251 cost = 0.04658979
Epoch: 1251 accuracy = 0.67954910
Epoch: 1252 cost = 0.04551888
Epoch: 1252 accuracy = 0.68921095
Epoch: 1253 cost = 0.04395443
Epoch: 1253 accuracy = 0.69645733
Epoch: 1254 cost = 0.04323980
Epoch: 1254 accuracy = 0.70933980
Epoch: 1255 cost = 0.04319318
Epoch: 1255 accuracy = 0.71336555
Epoch: 1256 cost = 0.04446544
Epoch: 1256 accuracy = 0.71014494
Epoch: 1257 cost = 0.04456747
Epoch: 1257 accuracy = 0.71175522
Epoch: 1258 cost = 0.04374945
Epoch: 1258 accuracy = 0.70692432
Epoch: 1259 cost = 0.04409768
Epoch: 1259 accuracy = 0.69243157
Epoch: 1260 cost = 0.04685169
Epoch: 1260 accuracy = 0.66908211
Epoch: 1261 cost = 0.04675683
Epoch: 1261 accuracy = 0.68921095
Epoch: 1262 cost = 0.04382106
Epoch: 1262 accuracy = 0.70692432
Epoch:

Epoch: 1420 cost = 0.03996264
Epoch: 1420 accuracy = 0.72544283
Epoch: 1421 cost = 0.03956512
Epoch: 1421 accuracy = 0.72061193
Epoch: 1422 cost = 0.03952410
Epoch: 1422 accuracy = 0.71175522
Epoch: 1423 cost = 0.04101530
Epoch: 1423 accuracy = 0.68760067
Epoch: 1424 cost = 0.04474873
Epoch: 1424 accuracy = 0.68840581
Epoch: 1425 cost = 0.04192263
Epoch: 1425 accuracy = 0.71175522
Epoch: 1426 cost = 0.03939599
Epoch: 1426 accuracy = 0.71578097
Epoch: 1427 cost = 0.03927971
Epoch: 1427 accuracy = 0.72705317
Epoch: 1428 cost = 0.03990816
Epoch: 1428 accuracy = 0.72383255
Epoch: 1429 cost = 0.03983515
Epoch: 1429 accuracy = 0.72463769
Epoch: 1430 cost = 0.03940824
Epoch: 1430 accuracy = 0.72383255
Epoch: 1431 cost = 0.03927162
Epoch: 1431 accuracy = 0.71256042
Epoch: 1432 cost = 0.04023479
Epoch: 1432 accuracy = 0.69645733
Epoch: 1433 cost = 0.04414412
Epoch: 1433 accuracy = 0.68438005
Epoch: 1434 cost = 0.04255312
Epoch: 1434 accuracy = 0.70370370
Epoch: 1435 cost = 0.03940388
Epoch: 143

Epoch: 1581 cost = 0.03581451
Epoch: 1581 accuracy = 0.72866344
Epoch: 1582 cost = 0.03663314
Epoch: 1582 accuracy = 0.73107892
Epoch: 1583 cost = 0.03629461
Epoch: 1583 accuracy = 0.72946858
Epoch: 1584 cost = 0.03577227
Epoch: 1584 accuracy = 0.72785830
Epoch: 1585 cost = 0.03580005
Epoch: 1585 accuracy = 0.72302735
Epoch: 1586 cost = 0.03770657
Epoch: 1586 accuracy = 0.70048308
Epoch: 1587 cost = 0.04071504
Epoch: 1587 accuracy = 0.70450884
Epoch: 1588 cost = 0.03770055
Epoch: 1588 accuracy = 0.72785830
Epoch: 1589 cost = 0.03556175
Epoch: 1589 accuracy = 0.72866344
Epoch: 1590 cost = 0.03564877
Epoch: 1590 accuracy = 0.73027372
Epoch: 1591 cost = 0.03646949
Epoch: 1591 accuracy = 0.73188406
Epoch: 1592 cost = 0.03604726
Epoch: 1592 accuracy = 0.73027372
Epoch: 1593 cost = 0.03553630
Epoch: 1593 accuracy = 0.72946858
Epoch: 1594 cost = 0.03564591
Epoch: 1594 accuracy = 0.72061193
Epoch: 1595 cost = 0.03788475
Epoch: 1595 accuracy = 0.70048308
Epoch: 1596 cost = 0.04041993
Epoch: 159

Epoch: 1757 cost = 0.03119674
Epoch: 1757 accuracy = 0.74798709
Epoch: 1758 cost = 0.03121983
Epoch: 1758 accuracy = 0.73510468
Epoch: 1759 cost = 0.03272027
Epoch: 1759 accuracy = 0.73590982
Epoch: 1760 cost = 0.03234599
Epoch: 1760 accuracy = 0.74154592
Epoch: 1761 cost = 0.03123551
Epoch: 1761 accuracy = 0.73913044
Epoch: 1762 cost = 0.03208589
Epoch: 1762 accuracy = 0.72463769
Epoch: 1763 cost = 0.03524490
Epoch: 1763 accuracy = 0.71980679
Epoch: 1764 cost = 0.03466350
Epoch: 1764 accuracy = 0.73429954
Epoch: 1765 cost = 0.03193277
Epoch: 1765 accuracy = 0.74396133
Epoch: 1766 cost = 0.03088992
Epoch: 1766 accuracy = 0.74637681
Epoch: 1767 cost = 0.03121949
Epoch: 1767 accuracy = 0.73349434
Epoch: 1768 cost = 0.03274543
Epoch: 1768 accuracy = 0.73913044
Epoch: 1769 cost = 0.03181755
Epoch: 1769 accuracy = 0.74154592
Epoch: 1770 cost = 0.03100113
Epoch: 1770 accuracy = 0.73590982
Epoch: 1771 cost = 0.03235155
Epoch: 1771 accuracy = 0.72383255
Epoch: 1772 cost = 0.03541614
Epoch: 177

Epoch: 1939 accuracy = 0.72463769
Epoch: 1940 cost = 0.03106797
Epoch: 1940 accuracy = 0.73752016
Epoch: 1941 cost = 0.02863377
Epoch: 1941 accuracy = 0.74718195
Epoch: 1942 cost = 0.02742784
Epoch: 1942 accuracy = 0.74879229
Epoch: 1943 cost = 0.02796277
Epoch: 1943 accuracy = 0.73107892
Epoch: 1944 cost = 0.02959123
Epoch: 1944 accuracy = 0.74315619
Epoch: 1945 cost = 0.02799524
Epoch: 1945 accuracy = 0.75040257
Epoch: 1946 cost = 0.02726128
Epoch: 1946 accuracy = 0.74154592
Epoch: 1947 cost = 0.02857896
Epoch: 1947 accuracy = 0.72544283
Epoch: 1948 cost = 0.03159994
Epoch: 1948 accuracy = 0.72866344
Epoch: 1949 cost = 0.03035388
Epoch: 1949 accuracy = 0.73752016
Epoch: 1950 cost = 0.02811821
Epoch: 1950 accuracy = 0.75120771
Epoch: 1951 cost = 0.02720770
Epoch: 1951 accuracy = 0.74718195
Epoch: 1952 cost = 0.02826534
Epoch: 1952 accuracy = 0.73268920
Epoch: 1953 cost = 0.02930039
Epoch: 1953 accuracy = 0.74557167
Epoch: 1954 cost = 0.02747018
Epoch: 1954 accuracy = 0.75120771
Epoch:

In [21]:
# on 1000 epoch
print(sess.run([accuracy, label_pred], feed_dict={X: X_train[1:2], Y: y_train[1:2]}))
print(sess.run([accuracy, label_pred], feed_dict={X: X_train[101:102], Y: y_train[101:102]}))
print(sess.run([accuracy], feed_dict={X: X_test1, Y: y_test1}))

[1.0, array([[1.1080623e-05, 9.1624629e-01, 2.1740220e-13, 4.7239296e-13,
        1.0100548e-12, 5.3940469e-04, 1.1220617e-07, 1.3386392e-15,
        8.3203182e-02, 3.4613764e-14, 1.5797573e-10, 1.6044227e-15,
        5.3729293e-10, 8.2170830e-34]], dtype=float32)]
[1.0, array([[9.3339430e-23, 1.2510322e-14, 3.4096114e-12, 1.4632762e-06,
        2.2841898e-04, 2.3446327e-04, 3.8945657e-20, 1.3721193e-04,
        2.3118316e-06, 9.8091386e-06, 1.8940601e-02, 9.7525626e-01,
        5.1894872e-03, 1.8408294e-11]], dtype=float32)]
[0.67464113]


In [47]:
# on 1000 epoch
a = X_train[0].reshape(1,10)
a1 = y_train[0].reshape(1,14)
b = X_train[600].reshape(1,10)
b1 = y_train[600].reshape(1,14)
print(a.shape)
print(sess.run(accuracy, feed_dict={X: a, Y: a1}))
print(sess.run(accuracy, feed_dict={X: b, Y: b1}))

(1, 10)
1.0
0.0
